In [1]:
# example of training an unstable gan for generating a handwritten digit
from os import makedirs
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from matplotlib import pyplot

# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # define model
    model = Sequential()
    # downsample to 14x14
    model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init,
    input_shape=in_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 7x7
    model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # define model
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    # upsample to 14x14
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 28x28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    # output 28x28x1
    model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# load mnist images
def load_real_samples():
    # load dataset
    (trainX, trainy), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # select all of the examples for a given class
    selected_ix = trainy == 8
    X = X[selected_ix]
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

# # select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # select images
    X = dataset[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n_samples, 1))
    return X, y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    # prepare fake examples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(10 * 10):
        # define subplot
        pyplot.subplot(10, 10, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to file
    pyplot.savefig('results_collapse/generated_plot_%03d.png' % (step+1))
    pyplot.close()
    # save the generator model
    g_model.save('results_collapse/model_%03d.h5' % (step+1))
    
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
    # plot loss
    pyplot.subplot(2, 1, 1)
    pyplot.plot(d1_hist, label='d-real')
    pyplot.plot(d2_hist, label='d-fake')
    pyplot.plot(g_hist, label='gen')
    pyplot.legend()
    # plot discriminator accuracy
    pyplot.subplot(2, 1, 2)
    pyplot.plot(a1_hist, label='acc-real')
    pyplot.plot(a2_hist, label='acc-fake')
    pyplot.legend()
    # save plot to file
    pyplot.savefig('results_collapse/plot_line_plot_loss.png')
    pyplot.close()
    
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
    # calculate the number of batches per epoch
    bat_per_epo = int(dataset.shape[0] / n_batch)
    # calculate the total iterations based on batch and epoch
    n_steps = bat_per_epo * n_epochs
    # calculate the number of samples in half a batch
    half_batch = int(n_batch / 2)
    # prepare lists for storing stats each iteration
    d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
    # manually enumerate epochs
    for i in range(n_steps):
        # get randomly selected ✬real✬ samples
        X_real, y_real = generate_real_samples(dataset, half_batch)
        
        # update discriminator model weights
        for j in range(2):
            d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
        
        # generate ✬fake✬ examples
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        
        # update discriminator model weights
        for j in range(2):
            d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
        
        # prepare points in latent space as input for the generator
        X_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        
        # update the generator via the discriminator✬s error
        for j in range(3): # Generator updated twice per each discriminator update (to avoid convergence failure)
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
        
        # summarize loss on this batch
        print('>%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
                    (i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
        # record history
        d1_hist.append(d_loss1)
        d2_hist.append(d_loss2)
        g_hist.append(g_loss)
        a1_hist.append(d_acc1)
        a2_hist.append(d_acc2)
        # evaluate the model performance every ✬epoch✬
        if (i+1) % bat_per_epo == 0:
            summarize_performance(i, g_model, latent_dim)
    plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)

# make folder for results
makedirs('results_collapse', exist_ok=True)
# size of the latent space
latent_dim = 1
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

Using TensorFlow backend.


(5851, 28, 28, 1)


C:\Users\Admin\anaconda3\envs\gputest\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Admin\anaconda3\envs\gputest\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, d1=0.354, d2=0.705 g=0.015, a1=90, a2=39


C:\Users\Admin\anaconda3\envs\gputest\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>2, d1=0.049, d2=0.868 g=0.005, a1=100, a2=0
>3, d1=0.033, d2=0.867 g=0.005, a1=100, a2=0
>4, d1=0.023, d2=0.814 g=0.005, a1=100, a2=0
>5, d1=0.020, d2=0.755 g=0.005, a1=100, a2=0
>6, d1=0.020, d2=0.707 g=0.006, a1=100, a2=18
>7, d1=0.013, d2=0.667 g=0.004, a1=100, a2=89
>8, d1=0.017, d2=0.628 g=0.004, a1=100, a2=100
>9, d1=0.015, d2=0.582 g=0.004, a1=100, a2=100
>10, d1=0.015, d2=0.545 g=0.003, a1=100, a2=100
>11, d1=0.014, d2=0.483 g=0.003, a1=100, a2=100
>12, d1=0.017, d2=0.415 g=0.003, a1=100, a2=100
>13, d1=0.017, d2=0.360 g=0.003, a1=100, a2=100
>14, d1=0.016, d2=0.317 g=0.003, a1=100, a2=100
>15, d1=0.015, d2=0.267 g=0.002, a1=100, a2=100
>16, d1=0.010, d2=0.222 g=0.002, a1=100, a2=100
>17, d1=0.015, d2=0.184 g=0.002, a1=100, a2=100
>18, d1=0.009, d2=0.154 g=0.002, a1=100, a2=100
>19, d1=0.010, d2=0.131 g=0.002, a1=100, a2=100
>20, d1=0.011, d2=0.111 g=0.002, a1=100, a2=100
>21, d1=0.010, d2=0.092 g=0.002, a1=100, a2=100
>22, d1=0.006, d2=0.082 g=0.001, a1=100, a2=100
>23, d1=0.

>172, d1=0.002, d2=0.002 g=5.571, a1=100, a2=100
>173, d1=0.001, d2=0.002 g=5.784, a1=100, a2=100
>174, d1=0.003, d2=0.001 g=5.843, a1=100, a2=100
>175, d1=0.002, d2=0.002 g=5.788, a1=100, a2=100
>176, d1=0.001, d2=0.002 g=6.188, a1=100, a2=100
>177, d1=0.001, d2=0.001 g=6.084, a1=100, a2=100
>178, d1=0.001, d2=0.002 g=5.893, a1=100, a2=100
>179, d1=0.002, d2=0.002 g=5.861, a1=100, a2=100
>180, d1=0.001, d2=0.002 g=5.771, a1=100, a2=100
>181, d1=0.001, d2=0.002 g=4.901, a1=100, a2=100
>182, d1=0.001, d2=0.003 g=2.334, a1=100, a2=100
>183, d1=0.001, d2=0.008 g=2.126, a1=100, a2=100
>184, d1=0.001, d2=0.010 g=0.474, a1=100, a2=100
>185, d1=0.002, d2=0.000 g=4.472, a1=100, a2=100
>186, d1=0.091, d2=0.004 g=0.397, a1=98, a2=100
>187, d1=0.016, d2=0.001 g=3.951, a1=100, a2=100
>188, d1=0.094, d2=0.013 g=0.580, a1=98, a2=100
>189, d1=0.004, d2=0.004 g=1.802, a1=100, a2=100
>190, d1=0.020, d2=0.022 g=0.643, a1=100, a2=100
>191, d1=0.026, d2=0.001 g=4.123, a1=100, a2=100
>192, d1=0.048, d2=0.0

>343, d1=0.716, d2=0.449 g=1.312, a1=57, a2=96
>344, d1=0.599, d2=0.354 g=1.312, a1=67, a2=100
>345, d1=0.580, d2=0.440 g=1.418, a1=65, a2=100
>346, d1=0.667, d2=0.443 g=1.377, a1=62, a2=100
>347, d1=0.625, d2=0.414 g=1.035, a1=68, a2=96
>348, d1=0.563, d2=0.522 g=0.954, a1=67, a2=65
>349, d1=0.667, d2=0.577 g=1.329, a1=60, a2=90
>350, d1=0.737, d2=0.498 g=0.881, a1=56, a2=100
>351, d1=0.776, d2=0.684 g=1.207, a1=53, a2=62
>352, d1=0.881, d2=0.458 g=1.092, a1=39, a2=89
>353, d1=0.723, d2=0.575 g=1.104, a1=59, a2=60
>354, d1=0.975, d2=0.599 g=1.078, a1=40, a2=85
>355, d1=0.881, d2=0.480 g=1.009, a1=40, a2=79
>356, d1=0.566, d2=0.436 g=1.282, a1=71, a2=100
>357, d1=0.805, d2=0.561 g=1.088, a1=46, a2=82
>358, d1=0.834, d2=0.597 g=1.212, a1=51, a2=68
>359, d1=0.850, d2=0.530 g=1.044, a1=48, a2=100
>360, d1=0.906, d2=0.718 g=0.920, a1=37, a2=29
>361, d1=0.893, d2=0.488 g=0.866, a1=39, a2=100
>362, d1=0.899, d2=0.675 g=0.783, a1=45, a2=54
>363, d1=0.754, d2=0.602 g=0.815, a1=50, a2=81
>364, 